# Imports

In [1]:
from pyiron_contrib.tinybase.task import AbstractTask, FunctionTask, SeriesTask, LoopTask

/home/poul/pyiron/contrib/pyiron_contrib/__init__.py:9: UserWarning: pyiron module not found, importing Project from pyiron_base
  warnings.warn("pyiron module not found, importing Project from pyiron_base")


In [2]:
from pyiron_contrib.tinybase.executor import ProcessExecutor, BackgroundExecutor, Executor

In [3]:
import logging
logging.getLogger().setLevel(20)

In [4]:
import numpy as np

# Function Task

## Basic

In [9]:
def calc_fib(n):
    import time
    n1 = n2 = 1
    for i in range(n):
        time.sleep(.1)
        x = n1 + n2
        n1 = n2
        n2 = x
    return x

In [10]:
f = FunctionTask(calc_fib)

In [11]:
f.input.storage

DataContainer([])

In [12]:
f.input.args

[]

In [13]:
f.input.kwargs

{}

In [14]:
f.input.kwargs['n'] = 10

In [15]:
f.input.kwargs

{'n': 10}

In [16]:
f.execute()

(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7f9b5941ab00>)

## We can use an executor to distribute the task to any compute resource

### Directly in the foreground

In [18]:
exe = Executor().submit([f])

In [19]:
exe.run()

In [20]:
exe.status[0]

ReturnStatus(Code.DONE, None)

In [21]:
exe.output[0].result

144

### Do the same but in the background

In [22]:
f = FunctionTask(calc_fib)

In [23]:
f.input.kwargs['n'] = 100

In [57]:
exe = BackgroundExecutor(max_threads=1).submit([f])

In [58]:
exe.run()

In [59]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [60]:
exe.wait()

In [61]:
exe.output[0].result

927372692193078999176

### Do the same but in the background as process

In [36]:
f = FunctionTask(calc_fib)

In [37]:
f.input.kwargs['n'] = 100

In [40]:
exe = ProcessExecutor(max_processes=1).submit([f])

In [41]:
exe.run()

In [43]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [44]:
exe.wait()

In [45]:
exe.output[0].result

927372692193078999176

In [46]:
exe._run_machine.state

<Code.FINISHED: 'finished'>

# Executors handle single Tasks and lists of them on the same footing

In [47]:
tasks = [FunctionTask(calc_fib) for _ in range(10)]

In [48]:
for i, n in enumerate(tasks):
    n.input.kwargs['n'] = 3 + i

## With the basic executor

In [49]:
exe = Executor().submit(tasks)
exe.run()

In [50]:
exe.output

(<pyiron_contrib.tinybase.task.FunctionOutput at 0x7f9b596ac160>,
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7f9b59462590>)

In [51]:
exe.output[1].result

8

## With the process executor

In [62]:
exe = ProcessExecutor(max_processes=4).submit(tasks)
exe.run()

In [63]:
exe.wait()

In [64]:
exe.status

[ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None)]

In [65]:
exe.output

In [66]:
exe.output[5].result

55

# SeriesTask

In [67]:
s = SeriesTask()

In [68]:
f1 = FunctionTask(calc_fib)

In [69]:
f2 = FunctionTask(np.sqrt)

In [70]:
def transfer(input, output):
    input.args = [output.result]

In [71]:
s.input.first(f1).then(f2, transfer)

In [72]:
s.input.tasks[0].input.kwargs['n'] = 10

In [73]:
status, output = s.execute()

In [74]:
status

ReturnStatus(Code.DONE, None)

In [75]:
output.result

144

# Loop Task

## Simple repeat loop

In [76]:
l = LoopTask()

In [77]:
l.input.task = FunctionTask(lambda: np.random.rand())

In [78]:
l.input.repeat(10, restart=lambda output, input, scratch: print(output.result))

In [80]:
l.execute()

0.6362656980328528
0.9334281883854404
0.6924270609164432
0.6860250441744892
0.022525309153174855
0.7312372220390315
0.48944745224752595
0.4136240195901667
0.2259678116613234


(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7f9b596a3400>)

## Loop with a termination condition

In [89]:
l = LoopTask()

In [90]:
l.input.task = FunctionTask(lambda: np.random.rand())

In [102]:
l.input.control_with(
    condition=lambda task, output, scratch: output.result < .15,
    restart=lambda output, input, scratch: print(output.result)
)

In [107]:
l.execute()

0.4416017514257269
0.3160638768343853
0.20690423045422135
0.5952022105132233
0.3844701289093476
0.996852574386064


(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7f9b5967d3f0>)